In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import math
import pdb
import sklearn as sk
from cleandata import cleanData
from sklearn.model_selection import train_test_split

In [2]:
store_CSV = pd.read_csv('data/store.csv')
train_CSV = pd.read_csv('data/train.csv', low_memory=False)

In [3]:
data = train_CSV.join(store_CSV.set_index('Store'), on='Store')

In [4]:
train_raw, test_raw = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
train_raw.shape, test_raw.shape

((510219, 18), (127555, 18))

In [6]:
print('Clean Train Data')
train = cleanData(train_raw, 'train')

Clean Train Data
Dropped rows without store-ids


/home/mirko/anaconda3/envs/dsr/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mirko/anaconda3/envs/dsr/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Extracted year, month and day from Date
Extracted and reset day of week


/home/mirko/git/minicomp-rossman/cleandata.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sales'][i] = 0
/home/mirko/anaconda3/envs/dsr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Set Sales to 0 if customers are 0
Dropped 0-sales rows in df


/home/mirko/git/minicomp-rossman/cleandata.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Open'][i] = 1


Set Open = 1 if Sales > 0
Filled school holidays based on state holidays
Public Holidays updated


/home/mirko/git/minicomp-rossman/cleandata.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['StateHoliday'][i] = row2['StateHoliday'].values[0]


Finished regional stateholidays
finished remaining stateholidays
Adjusted open status of shops according to state holidays
Mean customers of test cleaning:758.7492748450405
Finished filling in empty customers cells
Mean Sales of training set = 6836.722219708965
Finish sales
Mean Competition Distance of training set = 5446.105182647453
PromoIntervals encoded
Store Type Encoded
State Holidays Encoded
Assortment Type Encoded
---Cleaning completed---
Dropped last leftovers
All done!


### Provisional Data for your test data cleaning
Mean Customers = 758.7492748450405 /
Mean Sales = 6836.722219708965 /
Mean Competition Distance = 5446.105182647453 /

In [ ]:
print('Clean Test Data')
test = cleanData(test_raw, 'test')

In [8]:
#Defining targets and labels
X_train = train.drop(['Sales'], axis=1).to_numpy()
y_train = train['Sales'].to_numpy()
X_test = test.drop(['Sales'], axis=1).to_numpy()
y_test = test['Sales'].to_numpy()

y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [9]:
#Checking shapes
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((410231, 29), (410231, 1), (102508, 29), (102508, 1))

In [10]:
#Setting our scoring function
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [11]:
# This is a basic benchmark
def model_base(params, X_train, y_train, X_test, y_test, epochs):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    bst = xgb.train(params, dtrain, evals=[(dtest, 'Test')], num_boost_round=epochs)
    
    
    
    test = xgb.DMatrix(X_test)
    y_h_test = bst.predict(test)
   
    train = xgb.DMatrix(X_train)
    y_h_train = bst.predict(train)
    
    print(y_h_test.shape)
    print(y_test.shape)

    return (metric(y_h_test, y_test), metric(y_h_train, y_train))

In [12]:
# This is a basic benchmark with tuned with max_depth set to 9, 'min_child_weight' : 1, 'gamma' : 0, learning_rate : 0.4, subsample: 0.9

epochs = 50
x_axis =np.arange(6, 10, 1)
train_axis= np.zeros_like(x_axis)
test_axis=np.zeros_like(x_axis)
i=0

for num_parallel_tree in range(30,31,1):
    
    print('num_parallel_tree :',num_parallel_tree)  
    
    params = {
   'booster' : 'gbtree',
   'colsample_bynode': 0.8,
   'learning_rate': 0.4,
   'max_depth': 9,
   'num_parallel_tree': num_parallel_tree,
   'objective': 'reg:squarederror',
   #'objective': 'reg:linear',
   'subsample': 0.9,
   'n_estimators' : 20,
   #'as_pandas': True,
   'tree_method': 'exact',
   'eval_metric': 'mae',
   'min_child_weight' : 1,
   'gamma' : 0

    }

    eval_test, eval_train = model_base(params, X_train, y_train, X_test, y_test, epochs)
    
    print('Test result: ', eval_test)
    print('Train result: ', eval_train)
    #train_axis[i]=eval_train
    #test_axis[i] =eval_test
    #i+=1
    
    
#eval_test, eval_train = model_base(params, X_train, y_train, X_test, y_test, epochs)


num_parallel_tree : 30
[0]	Test-mae:4109.73
[1]	Test-mae:2485.28
[2]	Test-mae:1550.02
[3]	Test-mae:1065.56
[4]	Test-mae:846.452
[5]	Test-mae:738.419
[6]	Test-mae:694.108
[7]	Test-mae:664.636
[8]	Test-mae:645.997
[9]	Test-mae:621.112
[10]	Test-mae:609.954
[11]	Test-mae:599.942
[12]	Test-mae:586.579
[13]	Test-mae:575.304
[14]	Test-mae:566.449
[15]	Test-mae:544.624
[16]	Test-mae:538.974
[17]	Test-mae:526.263
[18]	Test-mae:520.123
[19]	Test-mae:512.071
[20]	Test-mae:503.745
[21]	Test-mae:494.377
[22]	Test-mae:486.361
[23]	Test-mae:479.891
[24]	Test-mae:473.487
[25]	Test-mae:466.642
[26]	Test-mae:461.188
[27]	Test-mae:455.806
[28]	Test-mae:452.945
[29]	Test-mae:448.831
[30]	Test-mae:445.602
[31]	Test-mae:443.056
[32]	Test-mae:439.431
[33]	Test-mae:435.849
[34]	Test-mae:433.38
[35]	Test-mae:430.695
[36]	Test-mae:428.022
[37]	Test-mae:425.785
[38]	Test-mae:423.769
[39]	Test-mae:421.381
[40]	Test-mae:418.856
[41]	Test-mae:417.122
[42]	Test-mae:415.718
[43]	Test-mae:414.342
[44]	Test-mae:413.33